<a href="https://colab.research.google.com/github/0hee0/2022-HCI-Capstone/blob/main/model_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi

Sun Nov 27 17:23:35 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   33C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!gdown --id 16MIleqoIr1vYxlGk4GKnGmrsCPuWkkpT

In [ ]:
!unzip --qq ECG5000.zip

In [ ]:
!pip install arff2pandas==1.0.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement arff2pandas==1.0.1 (from versions: none)
ERROR: No matching distribution found for arff2pandas==1.0.1


In [ ]:
!pip install scipy

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install -q -U watermark

     |████████████████████████████████| 1.6 MB 34.5 MB/s 


In [ ]:
!pip install -qq -U pandas

In [ ]:
%reload_ext watermark
%watermark -v -p numpy,pandas,torch,arff2pandas

Python implementation: CPython
Python version       : 3.7.15
IPython version      : 7.9.0

numpy      : 1.21.6
pandas     : 1.3.5
torch      : 1.12.1+cu113
arff2pandas: not installed



In [ ]:
import torch

import copy
import numpy as np
import pandas as pd
import seaborn as sns
from pylab import rcParams
import matplotlib.pyplot as plt
from matplotlib import rc
from sklearn.model_selection import train_test_split

from torch import nn, optim

import torch.nn.functional as F
# from arff2pandas import a2p
from scipy.io.arff import loadarff


%matplotlib inline
%config InlineBackend.figure_format='retina'

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
class Encoder(nn.Module):

  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(Encoder, self).__init__()

    self.seq_len, self.n_features = seq_len, n_features
    self.embedding_dim, self.hidden_dim = embedding_dim, 2 * embedding_dim

    self.rnn1 = nn.LSTM(
      input_size=n_features,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    
    self.rnn2 = nn.LSTM(
      input_size=self.hidden_dim,
      hidden_size=embedding_dim,
      num_layers=1,
      batch_first=True
    )
    
    self.rnn3 = nn.LSTM(
      input_size=n_features,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )

  def forward(self, x):
    x = x.reshape((1, self.seq_len, self.n_features))

    x, (_, _) = self.rnn1(x)
    x, (hidden_n, _) = self.rnn2(x)

    return hidden_n.reshape((self.n_features, self.embedding_dim))

class Decoder(nn.Module):

  def __init__(self, seq_len, input_dim=64, n_features=1):
    super(Decoder, self).__init__()

    self.seq_len, self.input_dim = seq_len, input_dim
    self.hidden_dim, self.n_features = 2 * input_dim, n_features

    self.rnn1 = nn.LSTM(
      input_size=input_dim,
      hidden_size=input_dim,
      num_layers=1,
      batch_first=True
    )

    self.rnn2 = nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    
    self.rnn3 = nn.LSTM(
      input_size=input_dim,
      hidden_size=self.hidden_dim,
      num_layers=1,
      batch_first=True
    )
    self.output_layer = nn.Linear(self.hidden_dim, n_features)

  def forward(self, x):
    x = x.repeat(self.seq_len, self.n_features)
    x = x.reshape((self.n_features, self.seq_len, self.input_dim))

    x, (hidden_n, cell_n) = self.rnn1(x)
    x, (hidden_n, cell_n) = self.rnn2(x)
    x = x.reshape((self.seq_len, self.hidden_dim))

    return self.output_layer(x)

class RecurrentAutoencoder(nn.Module):

  def __init__(self, seq_len, n_features, embedding_dim=64):
    super(RecurrentAutoencoder, self).__init__()

    self.encoder = Encoder(seq_len, n_features, embedding_dim).to(device)
    self.decoder = Decoder(seq_len, embedding_dim, n_features).to(device)

  def forward(self, x):
    x = self.encoder(x)
    x = self.decoder(x)

    return x

In [ ]:
model = torch.load('model_maxabs-scaling.pth')
model = model.to(device)

#### Time Series Prediction: Normal vs Risky

In [ ]:
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

In [ ]:
with open('ECG5000_TRAIN.arff') as f:
  train_raw_data = loadarff(f)
  train = pd.DataFrame(train_raw_data[0])

In [ ]:
with open('ECG5000_TEST.arff') as f:
  test_raw_data = loadarff(f)
  test = pd.DataFrame(test_raw_data[0])

In [ ]:
df = train.append(test)
df = df.sample(frac=1.0)

In [ ]:
from sklearn.preprocessing import MaxAbsScaler

scaler = MaxAbsScaler().fit(df)
scaled_df = scaler.transform(df)

In [ ]:
new_columns = list(df.columns)
new_columns[-1] = 'target'

df = pd.DataFrame(scaled_df, columns=new_columns)

In [ ]:
df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att132,att133,att134,att135,att136,att137,att138,att139,att140,target
0,-0.174729,-0.108803,-0.435356,-0.824607,-0.873795,-0.853759,-0.724813,-0.522290,-0.429795,-0.226828,...,-0.190140,-0.120740,0.069979,0.165462,0.106353,0.098796,-0.059026,0.008594,0.089218,0.2
1,0.156903,0.045729,-0.237927,-0.414122,-0.585238,-0.643583,-0.586344,-0.452783,-0.406570,-0.405846,...,0.201829,0.238421,0.165216,0.156741,0.075709,0.051024,0.057948,0.146529,0.183073,0.2
2,-0.136698,-0.202411,-0.331257,-0.338963,-0.381994,-0.410114,-0.432478,-0.425085,-0.294649,-0.147435,...,-0.263153,-0.415034,-0.612486,-0.840370,-0.884683,-0.894347,-0.617379,-0.409260,-0.265487,0.4
3,0.124429,-0.220910,-0.617388,-0.758795,-0.915259,-0.839802,-0.598740,-0.427944,-0.399825,-0.122239,...,0.273173,0.422919,0.395806,0.430583,0.312983,0.238090,0.037479,-0.082839,0.014559,0.2
4,0.036147,-0.360966,-0.641809,-0.652767,-0.566495,-0.444408,-0.207861,-0.070466,-0.004840,0.123530,...,-0.129457,-0.130237,-0.177400,-0.172841,-0.191977,-0.157698,-0.132908,-0.242446,-0.562799,0.2


In [ ]:
df.head()

,att1,att2,att3,att4,att5,att6,att7,att8,att9,att10,...,att132,att133,att134,att135,att136,att137,att138,att139,att140,target
1001,1.469756,-1.048520,-3.394356,-4.254399,-4.162834,-3.822570,-3.003609,-1.799773,-1.500033,-1.025095,...,0.945178,1.275588,1.617218,1.580279,1.306195,1.351674,1.915517,1.672103,-1.039932,b'1'
2086,-1.998602,-3.770552,-4.267091,-4.256133,-3.515288,-2.554540,-1.699639,-1.566366,-1.038815,-0.425483,...,1.008577,1.024698,1.051141,1.015352,0.988475,1.050191,1.089509,1.465382,0.799517,b'1'
2153,-1.187772,-3.365038,-3.695653,-4.094781,-3.992549,-3.425381,-2.057643,-1.277729,-1.307397,-0.623098,...,1.085007,1.467196,1.413850,1.283822,0.923126,0.759235,0.932364,1.216265,-0.824489,b'1'
555,0.604969,-1.671363,-3.236131,-3.966465,-4.067820,-3.551897,-2.582864,-1.804755,-1.688151,-1.025897,...,0.545222,0.649363,0.986846,1.234495,1.280039,1.215985,1.617971,2.196543,0.023843,b'1'
205,-1.197203,-3.270123,-3.778723,-3.977574,-3.405060,-2.392634,-1.726322,-1.572748,-0.920075,-0.388731,...,0.828168,0.914338,1.063077,1.393479,1.469756,1.392281,1.144732,1.668263,1.734676,b'1'


In [ ]:
# CLASS_NORMAL
normal_df = df[df.target == 0.2].drop(labels='target', axis=1)
normal_df.shape

(2919, 140)

In [ ]:
abnormal_df = df[df.target != 0.2].drop(labels='target', axis=1)
abnormal_df.shape

(2081, 140)

In [ ]:
pvc_df = df[df.target == 0.4].drop(labels='target', axis=1)
pvc_df.shape

(1767, 140)

In [ ]:
train_df, val_df = train_test_split(
  normal_df,
  test_size=0.15,
  random_state=RANDOM_SEED
)

In [ ]:
val_df, test_df = train_test_split(
  val_df,
  test_size=0.33, 
  random_state=RANDOM_SEED
)

In [ ]:
# import package
from matplotlib import pyplot as plt
import numpy as np

# import file
test_id = input("Please input test id : ") # sample
file_name = test_id + '_recorded.txt'
f = open("./s-patch-ex/" + file_name, "r")
lines = f.readlines()

print("\n===================================")
print("Packet Count :", len(lines))
print("===================================")

Please input test id : 2022-11-27

Packet Count : 46761


In [ ]:
from sklearn.utils import resample

# extract ECG sample

count = 0
data = []
ecg_sample = []
ecg_sample_count = 0

index = 0
index_list = []
max_val = -1

one_cycle_list = []
resampled_one_cycle_list = []
start_index = -1
end_index = -1
ecg_per_hour = []
resampled_ecg_per_hour = []

for idx, line in enumerate(lines):
# Split Packet
    ecg_data = line[12:780] # ECG Packet range

    # 첫 1~2초에는 noise가 있기 때문에 제외!
    if len(ecg_per_hour) == 0 and count < 2:
      count += 1
      continue

    for idx in range(128):  # Sample count is 256 
        # split 3 bytes (extract 2 samples from 3 bytes)
        start = idx * 6
        end = idx * 6 + 6
        split_packet = ecg_data[start:end]

        first_packet = split_packet[0:2]
        middle_packet = split_packet[2:4]
        end_packet = split_packet[4:6]

        int_packet_1 = int(middle_packet[1] + first_packet, 16)
        int_packet_2 = int(end_packet + middle_packet[0], 16)

        ecg_sample.append(int_packet_1)
        ecg_sample.append(int_packet_2)

    print("\n===================================")
    print("Packet Count :", count)
    print("Sample Count :", len(ecg_sample))
    print("===================================")

    count += 1
    if count == 3600: # 60초만 count
        # # TODO: 극댓값과 극댓값 사이의 값들 추출
        varCount = 0
        varHold = np.zeros(len(ecg_sample))
        flag = np.zeros(len(ecg_sample))

        Thres = 2300

        for i in range(len(ecg_sample)):
          if ecg_sample[i] < Thres:
            flag[i] = 1
          else:
            flag[i] = 0

          varHold[i] = varHold[i-1] + flag[i]

          if varHold[i] == varHold[i-1] and varHold[i] > 40:
            print('index:',i)
            index_list.append(i)

            varCount += 1
            varHold[i] = 0

        for i in range(len(index_list)):
          if i == 0:
            continue
          one_cycle = ecg_sample[index_list[i-1]:index_list[i]]
          # print(len(one_cycle))
          try:
            resampled_ecg = resample(one_cycle, n_samples=140, stratify=one_cycle, random_state=0)
            one_cycle_list.append(one_cycle)
            resampled_one_cycle_list.append(resampled_ecg)
          except:
            print("end")
            continue

        ecg_per_hour.append(one_cycle_list)
        resampled_ecg_per_hour.append(resampled_one_cycle_list)

        count = 0
        ecg_sample = []
        one_cycle_list = []
        resampled_one_cycle_list = []

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.

Packet Count : 2561
Sample Count : 655872

Packet Count : 2562
Sample Count : 656128

Packet Count : 2563
Sample Count : 656384

Packet Count : 2564
Sample Count : 656640

Packet Count : 2565
Sample Count : 656896

Packet Count : 2566
Sample Count : 657152

Packet Count : 2567
Sample Count : 657408

Packet Count : 2568
Sample Count : 657664

Packet Count : 2569
Sample Count : 657920

Packet Count : 2570
Sample Count : 658176

Packet Count : 2571
Sample Count : 658432

Packet Count : 2572
Sample Count : 658688

Packet Count : 2573
Sample Count : 658944

Packet Count : 2574
Sample Count : 659200

Packet Count : 2575
Sample Count : 659456

Packet Count : 2576
Sample Count : 659712

Packet Count : 2577
Sample Count : 659968

Packet Count : 2578
Sample Count : 660224

Packet Count : 2579
Sample Count : 660480

Packet Count : 2580
Sample Count : 660736

Packet Count : 2581
Sample Count : 660992

Packet Count : 2582
Sample Count : 661248

Packet Count : 25

In [ ]:
resampled_ecg_per_hour_list = resampled_ecg_per_hour

In [ ]:
print(len(resampled_ecg_per_hour_list))
print(len(resampled_ecg_per_hour_list[0]))
print(len(resampled_ecg_per_hour_list[0][1]))

12
4140
140


In [ ]:
ecg_data = pd.DataFrame(resampled_ecg_per_hour[0][:8])

In [ ]:
ecg_data = pd.DataFrame(resampled_ecg_per_hour[0])

In [ ]:
def create_dataset(df):

  sequences = df.astype(np.float32).to_numpy().tolist()

  dataset = [torch.tensor(s).unsqueeze(1).float() for s in sequences]

  n_seq, seq_len, n_features = torch.stack(dataset).shape

  return dataset, seq_len, n_features

In [ ]:
def create_scaled_dataset(df):
  scaler = MaxAbsScaler().fit(df)
  scaled_df = scaler.transform(df)

  sequences = scaled_df.astype(np.float32).tolist()

  dataset = [torch.tensor(s).unsqueeze(1).float() for s in sequences]

  n_seq, seq_len, n_features = torch.stack(dataset).shape

  return dataset, seq_len, n_features

In [ ]:
test_normal_dataset, _, _ = create_dataset(test_df)
test_abnormal_dataset, _, _ = create_dataset(abnormal_df)
test_pvc_dataset, _, _ = create_dataset(pvc_df)

In [ ]:
spatch_dataset, _, _ = create_scaled_dataset(ecg_data)

In [ ]:
new_dataset = [torch.sub(spatch_dataset[0], 1.1)]

In [ ]:
def predict(model, dataset):
  predictions, losses = [], []
  criterion = nn.L1Loss(reduction='sum').to(device)
  with torch.no_grad():
    model = model.eval()
    for seq_true in dataset:
      seq_true = seq_true.to(device)
      seq_pred = model(seq_true)

      loss = criterion(seq_pred, seq_true)

      predictions.append(seq_pred.cpu().numpy().flatten())
      losses.append(loss.item())
  return predictions, losses

In [ ]:
THRESHOLD = 200 # model 학습 결과를 보면, 12를 사용해야하지만 scaling 문제로 200으로 설정

In [ ]:
depression = [] # 하루 동안, 한 시간마다 우울감을 느꼈는지 여부를 저장하는 리스트

for resampled_ecg_per_hour in resampled_ecg_per_hour_list:
  ecg_data_per_hour = pd.DataFrame(resampled_ecg_per_hour)
  dataset_per_hour, _, _ = create_scaled_dataset(ecg_data_per_hour)

  pvc_count = 0

  for i, data in enumerate(dataset_per_hour):
    predictions, pred_losses = predict(model, [data])

    if pred_losses[0] >= THRESHOLD:
      pvc_count = pvc_count + 1
    
    if pvc_count >= 10:
      depression.append(True)
      break
  if pvc_count < 10:
    depression.append(False)

print(depression)

[True, True, False, True, True, True, False, False, False, True, False, False]


In [ ]:
sum(depression)

3